In [2]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

# Initialize Dash App
app = dash.Dash(__name__)

# Sample Years (Replace with your dataset's unique years)

df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')
# App Layout
app.layout = html.Div(children=[
    
    html.H1('Automobile Sales Statistics Dashboard', style = {'text-align': 'center','font-size': '35px', 'color': '#503D36'}),
    
    # Report Type Dropdown
    html.Label("Select Report Type:"),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder = 'Select a report type',
        value='Yearly Statistics',  # Default selection
        clearable=False,
        style = {'width' : '80%', 'font-size': '20px', 'padding' : '2px'}
    ),    
    html.Br(),
    html.Br(),   
    
    # Year Selection Dropdown
    html.Label("Select Year:"),
    dcc.Dropdown(
        id='year-drop',
        
        options=[{'label': i, 'value': i} for i in range(1980, 2024, 1)], ## passing years
        value= 1980,  # Default year
        placeholder = 'Select-year',
        clearable=False,
        style = {'font-size': '20px', 'padding' : '2px', 'width':'80%'}
    ),
    
    
    
    ### Adding Separate Division for output display -- Graphs
    
    html.Div([
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),])

])

# Callback to Enable/Disable Year Dropdown

@app.callback(
    [Output(component_id='output-container', component_property='children'),
     Output(component_id='year-drop', component_property='disabled')],
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='year-drop', component_property='value')]
)


def update_input_container(selected_report, selected_year):   ## This will disable based on selected report type
    try:

        """if selected_report is None or selected_year is None:
            return [html.Div(), False]"""

        if selected_report =='Recession Period Statistics': 
            recession_data = df[df['Recession'] == 1]

            yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
            R_chart_1 = dcc.Graph(
                    figure=px.line(yearly_rec, 
                        x='Year', y='Automobile_Sales',
                        title="Automobile_Sales"))
            #Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
            avg_vehicles = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()

            R_chart_2 = dcc.Graph(
                    figure=px.line(avg_vehicles, 
                        x='Vehicle_Type', y='Automobile_Sales',
                        title="average number of vehicles sold by vehicle type"))
            # Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
            exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()

            R_chart_3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                         values= 'Advertising_Expenditure', names= 'Vehicle_Type',
                         title="total expenditure share by vehicle type"
                        ))

            # Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
            unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()

            R_chart_4 = dcc.Graph(figure=px.bar(unemp_data,
            x='unemployment_rate', y='Automobile_Sales',
            color='Vehicle_Type',
            labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
            title='Effect of Unemployment Rate on Vehicle Type and Sales'))


            return [
                    html.Div(className='chart-grid', children=[html.Div(children=R_chart_1),html.Div(children=R_chart_2)],style={'display': 'flex'}),
                    html.Div(className='chart-grid', children=[html.Div(children=R_chart_3),html.Div(children=R_chart_4)],style={'display': 'flex'})
                    ], True



        elif (selected_year and selected_report== 'Yearly Statistics') :
            yearly_data = df[df['Year'] == selected_year]


            # Plot 1 :Yearly Automobile sales using line chart for the whole period.


            yas= yearly_data.groupby('Month')['Automobile_Sales'].mean().reset_index()
            Y_chart1 = dcc.Graph(figure=px.line(yas, x = 'Month', y = 'Automobile_Sales', title = 'Yearly Automobile sales'))

            mas=yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
            Y_chart2 = dcc.Graph(figure=px.line(mas,
                x='Month', y='Automobile_Sales',
                title='Total Monthly Automobile Sales'))

            # Plot bar chart for average number of vehicles sold during the given year

            avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
            Y_chart3 = dcc.Graph( figure=px.bar(avr_vdata, x =  'Vehicle_Type', y = 'Automobile_Sales',color='Vehicle_Type', title='Average Vehicles Sold by Vehicle Type in the year {}'.format(selected_year)))

            # Plot 4 Total Advertisement Expenditure for each vehicle using pie chart

            exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
            Y_chart4 = dcc.Graph(
                figure=px.pie(exp_data, 
                values='Advertising_Expenditure', names='Vehicle_Type',
                title='Total Advertisment Expenditure for Each Vehicle'))
            return [
                    html.Div(className='chart-grid', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display':'flex'}),
                    html.Div(className='chart-grid', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                    ], False
        else:
            return [html.Div(), False]
    except Exception as e:
        return [html.Div(f"An error occurred: {str(e)}"), False]

    

# Run Server
if __name__ == '__main__':
    app.run_server( port=8091)



In [10]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Initialize Dash App
app = dash.Dash(__name__)

# Load data
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# App Layout
app.layout = html.Div([
    html.H1('Automobile Sales Statistics Dashboard', style={'text-align': 'center', 'font-size': '35px', 'color': '#503D36'}),
    
    html.Label("Select Report Type:"),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        clearable=False,
        style={'width': '80%', 'font-size': '20px', 'padding': '2px'}
    ),
    html.Br(),
    
    html.Label("Select Year:"),
    dcc.Dropdown(
        id='year-drop',
        options=[{'label': i, 'value': i} for i in range(1980, 2024)],
        value=1980,
        clearable=False,
        style={'width': '80%', 'font-size': '20px', 'padding': '2px'}
    ),
    
    html.Br(),
    
    # Graphs Container
    html.Div(id='output-container', style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'gap': '20px'})
])

# Callback

@app.callback(
    [Output('output-container', 'children'),
     Output('year-drop', 'disabled')],
    [Input('dropdown-statistics', 'value'),
     Input('year-drop', 'value')]
)


def update_graphs(selected_report, selected_year):
    try:
        if selected_report == 'Recession Period Statistics':
            recession_data = df[df['Recession'] == 1]

            R_chart_1 = dcc.Graph(
                figure=px.line(recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index(),
                               x='Year', y='Automobile_Sales', title="Automobile Sales During Recession"))
            
            R_chart_2 = dcc.Graph(
                figure=px.bar(recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index(),
                              x='Vehicle_Type', y='Automobile_Sales', color = 'Vehicle_Type', title="Avg Vehicles Sold by Type"))

            R_chart_3 = dcc.Graph(
                figure=px.pie(recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index(),
                              values='Advertising_Expenditure', names='Vehicle_Type',
                              title="Ad Expenditure by Vehicle Type"))
            
            R_chart_4 = dcc.Graph(
                figure=px.bar(recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index(),
                              x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                              title="Unemployment Rate vs Sales"))

            return [R_chart_1, R_chart_2, R_chart_3, R_chart_4], True

        elif selected_report == 'Yearly Statistics' and selected_year:
            yearly_data = df[df['Year'] == selected_year]

            Y_chart1 = dcc.Graph(
                figure=px.line(yearly_data.groupby('Month')['Automobile_Sales'].mean().reset_index(),
                               x='Month', y='Automobile_Sales', title='Monthly Avg Sales'))

            Y_chart2 = dcc.Graph(
                figure=px.line(yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index(),
                               x='Month', y='Automobile_Sales', title='Total Monthly Sales'))

            Y_chart3 = dcc.Graph(
                figure=px.bar(yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index(),
                              x='Vehicle_Type', y='Automobile_Sales', color='Vehicle_Type',
                              title=f'Avg Vehicles Sold ({selected_year})'))

            Y_chart4 = dcc.Graph(
                figure=px.pie(yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index(),
                              values='Advertising_Expenditure', names='Vehicle_Type',
                              title='Ad Expenditure by Vehicle Type'))

            return [Y_chart1, Y_chart2, Y_chart3, Y_chart4], False

        return [html.Div()], False

    except Exception as e:
        return [html.Div(f"Error: {str(e)}")], False


# Run Server
if __name__ == '__main__':
    app.run_server(port=8091)


In [9]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Initialize Dash App
app = dash.Dash(__name__)

# Load data
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# App Layout
app.layout = html.Div([
    html.H1('Automobile Sales Statistics Dashboard', style={'text-align': 'center', 'font-size': '35px', 'color': '#503D36'}),
    
    html.Label("Select Report Type:"),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        clearable=False,
        style={'width': '80%', 'font-size': '20px', 'padding': '2px'}
    ),
    html.Br(),
    
    html.Label("Select Year:"),
    dcc.Dropdown(
        id='year-drop',
        options=[{'label': i, 'value': i} for i in range(1980, 2024)],
        value=1980,
        clearable=False,
        style={'width': '80%', 'font-size': '20px', 'padding': '2px'}
    ),
    
    html.Br(),
    
    # Graphs Container
    html.Div(id='output-container', style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'gap': '20px'})
])

# Callback

@app.callback(
    Output('year-drop', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_input_container(selected_report):
    return selected_report == 'Recession Period Statistics'

@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'),
     Input('year-drop', 'value')]
)
def update_graphs(selected_report, selected_year):
    try:
        if selected_report == 'Recession Period Statistics':
            recession_data = df[df['Recession'] == 1]

            R_chart_1 = dcc.Graph(
                figure=px.line(recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index(),
                               x='Year', y='Automobile_Sales', title="Automobile Sales During Recession"))
            
            R_chart_2 = dcc.Graph(
                figure=px.bar(recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index(),
                              x='Vehicle_Type', color = 'Vehicle_Type', y='Automobile_Sales', title="Avg Vehicles Sold by Type"))

            R_chart_3 = dcc.Graph(
                figure=px.pie(recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index(),
                              values='Advertising_Expenditure', names='Vehicle_Type',
                              title="Ad Expenditure by Vehicle Type"))
            
            R_chart_4 = dcc.Graph(
                figure=px.bar(recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index(),
                              x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                              title="Unemployment Rate vs Sales"))

            return [R_chart_1, R_chart_2, R_chart_3, R_chart_4]

        elif selected_report == 'Yearly Statistics' and selected_year:
            yearly_data = df[df['Year'] == selected_year]

            Y_chart1 = dcc.Graph(
                figure=px.line(yearly_data.groupby('Month')['Automobile_Sales'].mean().reset_index(),
                               x='Month', y='Automobile_Sales', title='Monthly Avg Sales'))

            Y_chart2 = dcc.Graph(
                figure=px.line(yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index(),
                               x='Month', y='Automobile_Sales', title='Total Monthly Sales'))

            Y_chart3 = dcc.Graph(
                figure=px.bar(yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index(),
                              x='Vehicle_Type', y='Automobile_Sales', color='Vehicle_Type',
                              title=f'Avg Vehicles Sold ({selected_year})'))

            Y_chart4 = dcc.Graph(
                figure=px.pie(yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index(),
                              values='Advertising_Expenditure', names='Vehicle_Type',
                              title='Ad Expenditure by Vehicle Type'))

            return [Y_chart1, Y_chart2, Y_chart3, Y_chart4]

        return [html.Div("Please select a valid report type and year.")]

    except Exception as e:
        return [html.Div(f"Error: {str(e)}")]


# Run Server
if __name__ == '__main__':
    app.run_server(port=8092)
